In [1]:
using StatsBase

In [2]:
struct Populacao
    n::Integer
    I0::Integer
    estadoInicial::Array{T} where T <: Integer
    residencia::Array{T} where T <: Integer
    bairro::Array{T} where T <: Integer
    social::Array{T} where T <: Integer
    posicao::Array{Float64, 2}
    estadoAtual::Array{T} where T <: Integer
end

struct ResumoPopulacao
    populacao::Populacao
    nResidencias::Integer
    residencias::Array
    nSociais::Integer
    sociais::Array
    nBairros::Integer
    bairros::Array
    distanciaBairral::Array{T, 2} where T <: Number
end

function selectiveRand(v)
    aux = zeros(length(v))
    aux[v] .= rand(sum(v))
    return aux
end

function rowWiseNorm(A)
    return sqrt.(sum(abs2, A, dims=2)[:, 1])
end

rowWiseNorm (generic function with 1 method)

In [3]:
function foo(resumoPop, suscetiveis, infectados, fKernel)
    aux = zeros(sum(suscetiveis))
    aux2 = (1:resumoPop.populacao.n)[suscetiveis]
    Threads.@threads for i in 1:length(sum(suscetiveis))
        aux[i] = calculaDistancia(resumoPop, i, infectados, fKernel)
    end
    return aux
end

function miniPassoMatricial(estados::Array{T} where T <: Integer)
    popSuscetiveis = estados.==1
    popInfectados = estados.==2
    
    n = length(estados)
    nInfectados = sum(popInfectados)
    nSuscetiveis = sum(popSuscetiveis)

    expostos = zeros(Bool, n, n)
    expostos[popSuscetiveis, popInfectados] .= true
    return sum(expostos, dims=2)[:, 1]
end

function passoMisto(resumoPop::ResumoPopulacao, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, 
        γ::Number, δ::Number, fKernel)
    """
        Entrada:
            resumoPop: 
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: probabilidade de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """
    pop = resumoPop.populacao
    popSuscetiveis = pop.estadoAtual.==1
    popInfectados = pop.estadoAtual.==2
    
    contatos = zeros(pop.n)
    Threads.@threads for i in resumoPop.residencias
        contatos[i] .+= miniPassoMatricial(pop.estadoAtual[i]) .* α[i]
    end
    
    Threads.@threads for i in resumoPop.sociais
        contatos[i] .+= miniPassoMatricial(pop.estadoAtual[i]) .* β[i]
    end
    
    #contatos .+= sum(resumoPop.globalKernel[:, popInfectados], dims=2)[:, 1] .* θ
    contatos[popSuscetiveis] .+= foo(resumoPop, popSuscetiveis, popInfectados, fKernel)
    
    prob = exp.(- δ .* contatos)
    
    novosInfectados = selectiveRand(popSuscetiveis) .> prob
    novosRecuperados = selectiveRand(popInfectados) .> γ
    
    infectados = ((popInfectados .& (.~novosRecuperados)) .| novosInfectados)
    suscetiveis = (popSuscetiveis .& (.~novosInfectados))

    return 3 .* ones(Int, pop.n) .- 2 .* suscetiveis .- infectados
end

function evolucaoMista(
        resumoPop::ResumoPopulacao, tempos::Array{T} where T <: Number, nSim::Integer, 
        α::Array{T} where T <: Number, β::Array{T} where T <: Number, θ::Array{T} where T <: Number, γ::Number, fKernel)
    """
        Entrada:
            resumoPop: 
            tempos:
            α: taxa de transmissão residencial
            β: taxa de transmissão social
            θ: taxa de transmissão global
            γ: parâmetro da exponencial de não recuperação
            δ: tamanho do passo temporal
            fKernel: ?
    """
    populacao = resumoPop.populacao
    
    nT = length(tempos)
    passos = tempos[2:end] - tempos[1:(end-1)]
    Γ = exp.(- γ .* passos)
    
    S = zeros(nSim, nT)
    I = zeros(nSim, nT)
    R = zeros(nSim, nT)
    
    S[:, 1] .= populacao.n - populacao.I0
    I[:, 1] .= populacao.I0
    
    for j in 1:nSim
        populacao.estadoAtual .= populacao.estadoInicial
        for (k, δ) in enumerate(passos)
            @time populacao.estadoAtual .= passoMisto(resumoPop, α, β, θ, Γ[k], δ, fKernel)
            
            S[j, k+1] = sum(populacao.estadoAtual .== 1)
            I[j, k+1] = sum(populacao.estadoAtual .== 2)
            R[j, k+1] = sum(populacao.estadoAtual .== 3)
        end
    end
    return S,I,R
end

evolucaoMista (generic function with 1 method)

In [4]:
using BenchmarkTools

In [5]:
function geraPopulacaoAleatoria(n, I0, nResidencias, nSociais, bairrosShape)
    pop0 = ones(Int, n)
    pop0[StatsBase.sample(1:n, I0, replace=false)] .= 2;
    residencia = rand(1:nResidencias, n)
    social = rand(1:nSociais, n)
    residenciasPos = rand(nResidencias, 2) .* bairrosShape'
    residenciasBai = [
        ceil(Int, residenciasPos[i, 1] / bairrosShape[1]) + 
        floor(Int, residenciasPos[i, 2] / bairrosShape[2]) * bairrosShape[2]
        for i in 1:nResidencias
    ]
    posicao = zeros(n, 2)
    bairro = zeros(Int, n)
    Threads.@threads for i in 1:n
        posicao[i, :] .= residenciasPos[residencia[i], :]
        bairro[i] = residenciasBai[residencia[i]]
    end
    return Populacao(n, I0, pop0, residencia, bairro, social, posicao, copy(pop0))
end

geraPopulacaoAleatoria (generic function with 1 method)

In [6]:
function calculaDistancia(resumoPop::ResumoPopulacao, i::Integer, infectados, fKernel)
    pop = resumoPop.populacao
    bairro = pop.bairro[i]
    aux = 0.
    aux += sum([sum(infectados[i]) for i in resumoPop.bairros] .* resumoPop.distanciaBairral[bairro, :])
    aux += sum(fKernel(pop.posicao[resumoPop.bairros[bairro], :] .- pop.posicao[i, :]'))
    return aux
end

calculaDistancia (generic function with 1 method)

In [7]:
function power_decay(a::Array)
    b = rowWiseNorm(a)
    return sum(b ./ (b .+ 1.5));
end

power_decay (generic function with 1 method)

In [8]:
function resumePopulacao(populacao::Populacao, fKernel)
    @time begin
    nResidencias = maximum(populacao.residencia)
    nSociais = maximum(populacao.social)
    nBairros = maximum(populacao.bairro)
    
    residencias = [(1:populacao.n)[populacao.residencia .== i] for i in 1:nResidencias]
    sociais = [(1:populacao.n)[populacao.social .== i] for i in 1:nSociais]
    bairros = [(1:populacao.n)[populacao.bairro .== i] for i in 1:nBairros]
    end
    
    @time centros = hcat([mean(populacao.posicao[i, :], dims=1) for i in bairros]...)
    @time dist = hcat([fKernel(centros .- centros[i, :]') for i in 1:nBairros]...)
    return ResumoPopulacao(populacao, nResidencias, residencias, nSociais, sociais, nBairros, bairros, dist)
end

resumePopulacao (generic function with 1 method)

In [12]:
n = 100000
I0 = 1000
nResidencias = 30000
nSociais = 500
populacao = geraPopulacaoAleatoria(n, I0, nResidencias, nSociais, [10,5]);

resumoPop = resumePopulacao(populacao, power_decay);

  1.092979 seconds (364.01 k allocations: 504.822 MiB, 1.91% gc time)
  0.000404 seconds (9 allocations: 1.526 MiB)
  0.000014 seconds (18 allocations: 1.031 KiB)


In [13]:
α = 0.1 * ones(n)
β = 1e-3 * ones(n)
θ = zeros(n);
@time Threads.@threads for i in 1:n
#     @time bairro = populacao.bairro[i]
#     @time aux = sum([length(i) for i in resumoPop.bairros] .* resumoPop.distanciaBairral[bairro, :])
#     @time aux += sum(power_decay(populacao.posicao[resumoPop.bairros[bairro], :] .- populacao.posicao[i, :]'))
#     @time θ[i] = 1 / aux
    θ[i] = 1 / calculaDistancia(resumoPop, i, ones(Bool, n), power_decay)
end
γ = 0.2;

 86.255387 seconds (5.82 M allocations: 614.865 GiB, 4.97% gc time)


In [15]:
@time a = evolucaoMista(resumoPop, Array(1:10), 1, α, β, θ, γ, power_decay);

  0.084646 seconds (946.28 k allocations: 86.785 MiB, 23.41% gc time)
  0.038287 seconds (946.28 k allocations: 86.761 MiB)
  0.033361 seconds (946.28 k allocations: 86.734 MiB, 17.93% gc time)
  0.068132 seconds (946.43 k allocations: 86.712 MiB, 22.54% gc time)
  0.022501 seconds (946.28 k allocations: 86.660 MiB)
  0.070339 seconds (946.43 k allocations: 86.623 MiB, 24.19% gc time)
  0.022346 seconds (946.28 k allocations: 86.561 MiB)
  0.066739 seconds (946.43 k allocations: 86.505 MiB, 25.02% gc time)
  0.022420 seconds (946.28 k allocations: 86.417 MiB)
  0.432840 seconds (8.52 M allocations: 780.238 MiB, 17.30% gc time)


In [54]:
a[1]

2×10 Array{Float64,2}:
 69300.0  68921.0  68432.0  67879.0  …  65528.0  64461.0  63191.0  61781.0
 69300.0  68981.0  68544.0  68009.0     65888.0  64954.0  63821.0  62574.0